In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

sys.path.insert(0, "..")

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import polars as pl

In [21]:
from tsdb_benchmarks.monetdb import MonetDB

db = MonetDB()


db.fetch("select * from test")

id,val,time,on_off,dbl
i32,f32,datetime[ns],bool,f64
1,1.0,null,true,null
2,null,null,true,null
3,2.0,null,true,null
22,null,2025-05-02 12:00:00,true,null
222,null,null,true,null
223,null,null,false,null
